# Heat of Formation Calculator using the QM9 Dataset
This notebook calculates the Heat of Formation for each molecule in the QM9 dataset and returns a CSV file of the dataset with the counts of the C, H, O, N, and F atoms and Heat of Formation.

## A General Introduction

### QM9 Dataset
The QM9 dataset is a collection of ~134,000 small organic molecules. It contains characteristic information including, but not limited to, their structure, homo/lumo energies, internal energy, enthalpies, energy of atomization, and heat capacity.<br /> You can find more information about the dataset here: https://www.nature.com/articles/sdata201422

### SMILES (Simplified molecular-input line-entry system)
While energetic and thermal properties are stored numerically, structural information about the molecules are stored as SMILES strings. In simple words, a SMILES string is a way to represent a molecule by only using ASCII characters.

Why? They're relatively easy for us (humans) to understand and write, while being a simple way to tell computers about chemical structures without using complex file formats.

Examples: <br/> 
• CO<sub>2</sub> --> O=C=O <br/>
• H<sub>2</sub> --> O <br/>
• CH<sub>4</sub> --> C <br/>
• N<sub>2</sub> --> N#N

For a more detailed reference, check this out: https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system

## Setting up
Before we dive into the code and chemistry, we need to do some housekeeping.

### Installing requirements
First, we need to get our python installation tool, `pip` (in case we don't already have it). Lines 1-2 of the following code block take care of that. <br/>
Next, we need a way to view and store our data. For this, we use `pandas`, an open source data analysis and manipulation tool. (line 3) Reference: https://pandas.pydata.org/docs/ <br/>
Finally, we need a way to actually read our SMILES strings from QM9. For that we will use `pysmiles`, a lightweight SMILES reader and writer. (line 4) Reference: https://pypi.org/project/pysmiles/

In [1]:
!curl -O https://raw.github.com/pypa/pip/master/contrib/get-pip.py
!python get-pip.py

!pip install pandas
!pip install pysmiles

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Setting up a Pandas DataFrame with the QM9 dataset
Now that we have all our libraries, let's load the QM9 dataset into a `DataFrame`. We're getting the dataset from this github repository: https://raw.githubusercontent.com/OpenDrugAI/AttentiveFP/master/data/qm9.csv <br/>
The last line of the following code block prints out the data stored in the `DataFrame` that we just created. This is going to be our first look at the QM9 dataset!

In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/OpenDrugAI/AttentiveFP/master/data/qm9.csv')
display(df)

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,...,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,...,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,...,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,...,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,...,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028
5,gdb_6,C=O,285.48839,38.982300,34.298920,2.1089,14.18,-0.2670,-0.0406,0.2263,...,0.026603,-114.483613,-114.480746,-114.479802,-114.505268,6.413,-358.756935,-360.512706,-362.291066,-340.464421
6,gdb_7,CC,80.46225,19.906490,19.906330,0.0000,23.95,-0.3385,0.1041,0.4426,...,0.074542,-79.764152,-79.760666,-79.759722,-79.787269,10.098,-670.788296,-675.710476,-679.860821,-626.927299
7,gdb_8,CO,127.83497,24.858720,23.978720,1.5258,16.97,-0.2653,0.0784,0.3437,...,0.051208,-115.679136,-115.675816,-115.674872,-115.701876,8.751,-481.106758,-484.355372,-487.319724,-450.124128
8,gdb_9,CC#C,160.28041,8.593230,8.593210,0.7156,28.78,-0.2609,0.0613,0.3222,...,0.055410,-116.609549,-116.605550,-116.604606,-116.633775,12.482,-670.268091,-673.980434,-677.537155,-631.346845
9,gdb_10,CC#N,159.03567,9.223270,9.223240,3.8266,24.45,-0.3264,0.0376,0.3640,...,0.045286,-132.718150,-132.714563,-132.713619,-132.742149,10.287,-589.812024,-592.893721,-595.857446,-557.125708


## Understanding pysmiles
As we learned earlier, `pysmiles` is a lightweight SMILES reader and writer. It allows us to get molecular information (like the constituent elements and their count) from a SMILES string. <br/>
In the following code block, we're going to see what's the atomic composition of the molecule associated with the SMILES string 'C#C' and how `pysmiles` gives that information back to us.

In [3]:
from pysmiles import read_smiles

smiles = 'C#C'
mol = read_smiles(smiles, explicit_hydrogen=True)

print(mol.nodes(data='element'))

[(0, 'C'), (1, 'C'), (2, 'H'), (3, 'H')]


Let's break down what we just saw. <br/>
`pysmiles` accepts the SMILES string as a parameter, also checking if we want to know about the hydrogens in the molecule. Once it has all that, it allows us to get all the elements that exist in the molecule in the form of an array of tuples. Each tuple contains the atom number (in 0 based indexing) and the atom's symbol. <br/>
Therefore, for ethene, we got [(0, 'C'), (1, 'C'), (2, 'H'), (3, 'H')], which is 2 carbon atoms and 2 hydrogen atoms. <br/>
Perfect! Now we know how `pysmiles` returns the information we need.

## Using pysmiles to count the number of C, H, O, N, and F atoms in a molecule
Using what we learned about pysmiles, let's create a function that tell us how many atoms of each of C, H, O, N, and F are in a molecule. <br/>
We are going to basically give `get_elements()` a SMILES string and it will return a dictionary with the counts of those 5 elements. It will do this by iterating through the list that pysmiles gives to us and simply adding 1 to the index associated with the element it bumps into.

In [4]:
def get_elements(smiles):
    elements = {'C': 0, 'H': 0, 'O': 0, 'N': 0, 'F': 0}
    
    mol = read_smiles(smiles, explicit_hydrogen=True)

    for e in mol.nodes(data='element'):
        elements[e[1]] = elements[e[1]] + 1
    
    return elements

elements = get_elements('C')
print(elements)


{'C': 1, 'H': 4, 'O': 0, 'N': 0, 'F': 0}


When we called `get_elements('C')`, we observe that the function returns a dictionary with index C with a value of 1, index H with a value 4, and everything else 0. This correctly captures the atomic composition of the CH<sub>4</sub> molecule (or methane) associated with the SMILES string 'C'. <br/>
Onwards!

## Calculating the Heat of Formation
Before we move any further, we need to first understand how to calculate the heat of formation of a given molecule. <br/>
To obtain the heat of formation of a molecule, i.e., the heat absorbed when 1 mole of a compound is formed, we need to find the heat of formation of the individual atoms minus the energy required to break the bonds of that molecule. <br/>
This is given by the following equation: \\[∆_fH^o(C_mH_n;298K) = \\]<span style="color:blue">\\[m∆_fH^o_{exptl}(C;298K) + n∆_fH^o_{exptl}(H;298K)-\\]</span><span style="color:green">\\[[mH^o_{calcd}(C;298K) + nH^o_{calcd}(H;298K)-H^o_{calcd}(C_mH_n;298K)]\\]</span><br/>
<i>Source: Saeys, Mark, et al. “Ab Initio Calculations for Hydrocarbons:&nbsp; Enthalpy of Formation, Transition State Geometry, and Activation Energy for Radical Reactions.” The Journal of Physical Chemistry A, vol. 107, no. 43, 2003, pp. 9147–9159., doi:10.1021/jp021706d.</i> <br/><br/>
The <span style="color:blue">blue</span> part of the equation is the experimentally calculated standard formation enthalpies of the constituent atoms, multiplied by the number of times they appear in the molecule. <span style="color:blue">QM9 does NOT contain this information.</span><br/>
The <span style="color:green">green</span> part of the equation is the amount of energy required to break the bonds in the molecule, or the atomization energy. <span style="color:green">QM9 contains this information.</span><br/><br/>

## Standard Enthalpies of Formation
In the following code block, we're defining a function `get_std_form_energy()` that accepts a dictionary with keys {'C', 'H', 'O', 'N', 'F'} and returns the standard enthalpy of formation for the given compositon. <br/>Values for the enthalpy of formation of each atom was found in “Wired Chemist.” Standard Enthalpies of Formation of Gaseous Atoms, www.wiredchemist.com/chemistry/data/enthalpies. 

In [5]:
def get_std_form_energy(elements):
    # in kcal/mol
    std_form_energies = {
        'C': 171.367,
        'H': 52.1033,
        'O': 59.5124,
        'N': 113.05,
        'F': 18.8815
        }
    
    energy = 0

    for e in elements:
        energy = energy + std_form_energies[e] * elements[e]
    
    return energy

print(get_std_form_energy(elements))


379.7802


`get_std_form_energy()` returns the final answer in kcal/mol

## Adding all this information to QM9
We are now ready to update QM9 with all the new information we can calculate/extract from our SMILES. 

First, we initialize a dictionary, where each key stores a list of associated data. Think of it like a column name associated with rows of data. <br/>
Then, we iterate through the QM9 `DataFrame` that we'd created earlier. For each molecule, we call `get_elements()` and update our dictionary with the number of C, H, O, N, and F atoms. We use this information with `get_stf_form_energy()` to get the experimental enthalpy.

Finally, we calculate the Heat of Formation for the molecule (FINALLY!). We subtract the atomization enthalpy (at 298 K) from the experimental enthalpy we just calculated. Et voila! We have our heat of formation for the molecule. <br/>
We repeat this procedure for all ~134,000 molecules in the `DataFrame` and then just add this new data to its own DataFrame.

NOTE: All the code that says `sys.stdout.write(...)` and `MAX_POINTS` is for a simple progress bar to keep track of how far along we are. A fun little thing to watch for the minute or so this thing takes to complete.

In [6]:
import sys

MAX_POINTS = len(df)

new_data = {'carbon': [], 'hydrogen': [], 'oxygen': [], 'nitrogen': [], 'fluorine': [], 'exp_enthalpy': [], 'hof': []}

for i in range(len(df)):
    molecule = df.loc[i, 'smiles']
    elements = get_elements(molecule)
    exp_enthalpy = get_std_form_energy(elements)
    
    new_data['carbon'].append(elements['C'])
    new_data['hydrogen'].append(elements['H'])
    new_data['oxygen'].append(elements['O'])
    new_data['nitrogen'].append(elements['N'])
    new_data['fluorine'].append(elements['F'])
    new_data['exp_enthalpy'].append(exp_enthalpy)
    
    # Calculating the HoF using internal energy at 298.15 K
    u298_atom = df.loc[i, 'u298_atom']
    new_data['hof'].append(u298_atom + exp_enthalpy)
    
    # A simple progress bar
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%% %d/%d" % ('#'*int(((i+1) / (MAX_POINTS/20))), float((i+1)/MAX_POINTS*100), (i+1), MAX_POINTS))
    sys.stdout.flush()

new_df = pd.DataFrame(new_data)    
display(new_df)

[####################] 100% 133885/133885

,carbon,hydrogen,oxygen,nitrogen,fluorine,exp_enthalpy,hof
0,1,4,0,0,0,379.7802,-18.863090
1,0,3,0,1,0,269.3599,-9.260371
2,0,2,1,0,0,163.7190,-50.255294
3,2,2,0,0,0,446.9406,59.702914
4,1,1,0,1,0,336.5203,33.613548
5,1,2,1,0,0,335.0860,-25.426706
6,2,6,0,0,0,655.3538,-20.356676
7,1,4,1,0,0,439.2926,-45.062772
8,3,4,0,0,0,722.5142,48.533766
9,2,3,0,1,0,612.0939,19.200179


### Appending the new information to the QM9 DataFrame
Here we simply append the new `DataFrame` that we had created to the QM9 `DataFrame`. We can finally take a look at what our complete dataset looks like :)

In [7]:
df = pd.concat([df, new_df], axis=1)
display(df)

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,u298_atom,h298_atom,g298_atom,carbon,hydrogen,oxygen,nitrogen,fluorine,exp_enthalpy,hof
0,gdb_1,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,...,-398.643290,-401.014647,-372.471772,1,4,0,0,0,379.7802,-18.863090
1,gdb_2,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,...,-278.620271,-280.399259,-259.338802,0,3,0,1,0,269.3599,-9.260371
2,gdb_3,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,...,-213.974294,-215.159658,-201.407171,0,2,1,0,0,163.7190,-50.255294
3,gdb_4,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,...,-387.237686,-389.016047,-365.800724,2,2,0,0,0,446.9406,59.702914
4,gdb_5,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,...,-302.906752,-304.091489,-288.720028,1,1,0,1,0,336.5203,33.613548
5,gdb_6,C=O,285.48839,38.982300,34.298920,2.1089,14.18,-0.2670,-0.0406,0.2263,...,-360.512706,-362.291066,-340.464421,1,2,1,0,0,335.0860,-25.426706
6,gdb_7,CC,80.46225,19.906490,19.906330,0.0000,23.95,-0.3385,0.1041,0.4426,...,-675.710476,-679.860821,-626.927299,2,6,0,0,0,655.3538,-20.356676
7,gdb_8,CO,127.83497,24.858720,23.978720,1.5258,16.97,-0.2653,0.0784,0.3437,...,-484.355372,-487.319724,-450.124128,1,4,1,0,0,439.2926,-45.062772
8,gdb_9,CC#C,160.28041,8.593230,8.593210,0.7156,28.78,-0.2609,0.0613,0.3222,...,-673.980434,-677.537155,-631.346845,3,4,0,0,0,722.5142,48.533766
9,gdb_10,CC#N,159.03567,9.223270,9.223240,3.8266,24.45,-0.3264,0.0376,0.3640,...,-592.893721,-595.857446,-557.125708,2,3,0,1,0,612.0939,19.200179


### Convert the updated QM9 DataFrame into a CSV file
For external use or reference, it is a good idea to export our `DataFrame` to a CSV (comma separated values) file. You will find this file in whichever directory this Jupyter notebook is. If you want it elsewhere, feel free to change the destination directory.

In [8]:
df.to_csv('qm9_HoF.csv')

And that's all for today, folks! You successfully walked through using one of the largest-machine-learning-relevant organic molecule datasets, broke down some SMILES, learned how to calculate the heat of formation of a molecule using easily available information and valuable data in QM9, and then updated said dataset with ALL this new information.

Cheerio :)